### Building a Local Knowledge Assistant with LangChain & OpenAI

In this project, I implemented a `Retrieval-Augmented Generation (RAG)` pipeline using `LangChain`, `Chroma`, and `OpenAI embeddings`. The system retrieves relevant chunks of information from my document collection to provide more accurate and context-aware answers. By combining retrieval with generation, I can leverage a smaller, pre-trained LLM while still achieving detailed and precise responses, without the need for costly fine-tuning. This setup allows me to experiment with AI-driven Q&A in a practical, hands-on way.

This project was inspired by [RAG + Langchain Python Project: Easy AI/Chat For Your Docs](https://www.youtube.com/watch?v=tcqEUSNCn8I) and adapted for personal learning and experimentation.


### Why RAG Matters?

As language models grow larger, updating and controlling their knowledge becomes increasingly impractical through fine-tuning alone. `RAG` introduces a retrieval-based paradigm that preserves model generality while enabling precise, real-time knowledge integration. &nbsp;&nbsp; ⇾(Expand the section for details)

**Traditional Approaches** <br>
**`Fine-tuning`** works but comes with serious baggage: massive **compute requirements**, maintaining multiple **model versions**, knowledge that goes **stale** (requiring full retraining), and catastrophic **forgetting** where models lose general capabilities. **`LoRA`** makes this more efficient by training **only low-rank matrices** instead of full weights, but you're still **burning GPU hours**, tuning **hyperparameters**, and—critically—**baking knowledge** into opaque model weights. Want to update a single fact? Good luck auditing which training examples influenced what, or making changes without retraining from scratch.

**RAG: Knowledge Without Retraining** <br>
`RAG` flips the script. Instead of modifying the model, it retrieves **relevant context** from **vector databases at inference time**. The advantages are game-changing: **instant knowledge updates** without retraining, **traceable citations** grounding outputs in source material, and **zero training compute**—just embed and index your documents. Your knowledge stays **external** and **auditable**, perfect for rapidly changing domains like customer support or compliance where information updates weekly. Plus, you get clean separation of concerns: **the LLM handles reasoning while retrieval manages knowledge**.

**Trade-offs** <br>
The catch? You're now in the **retrieval** game. Chunking strategies, embedding quality, and similarity metrics directly impact what context reaches your model. Poor retrieval means **hallucinations**. **Context windows** limit how much you can include, requiring **smart ranking** when relevant info spans many documents. But these challenges beat managing expensive retraining pipelines—RAG trades GPU clusters for vector database tuning, a much more tractable problem.

### Why LangChain?

LLM applications in production need to integrate with dozens of **services—vector databases**, **document loaders**, **model providers**, **monitoring tools**. Alternatives like `Mirascope` or `LiteLLM` focus on **specific integration** challenges, but `LangChain`'s ecosystem is unmatched: **100+ integrations** maintained by both the core team and community, reducing custom boilerplate significantly.  &nbsp;&nbsp; ⇾(Expand the section for details)

**Going Solo: Reinventing the Wheel** <br>
**Going solo means building infrastructure from scratch**. `Raw API` calls seem simple until you're managing conversation history, implementing retry logic, handling streaming responses, and coordinating multi-step chains. **Custom frameworks give you control but demand serious engineering**: designing abstractions, integrating new model providers, building observability, maintaining compatibility. Every `RAG` pipeline or `agent system` requires **solving the same orchestration problems** teams everywhere have already tackled.

**What LangChain Brings to the Table** <br>
`LangChain` consolidates years of collective learning into **reusable patterns**. The framework shines in three areas: **provider abstraction** (switch between OpenAI, Anthropic, local models without rewriting logic), **composable chains** (LCEL makes complex workflows readable and debuggable), and **ecosystem integrations** (vector stores, document loaders, agent tools—all plug-and-play). You're not just getting code—you're getting architectural patterns that scale, from simple prompt chains to sophisticated agent loops with memory and tool use.

**What You're Trading For: Velocity vs Transparency** <br>
The tradeoff is **framework dependency**. `LangChain` adds abstraction layers that can **obscure what's happening under the hood**. Simple tasks might feel overengineered, and rapid framework evolution means staying current with breaking changes. Debugging requires understanding LangChain's execution model, not just your application logic. But the math usually works out: teams shipping complex LLM features fast versus teams stuck building plumbing. For most production use cases, LangChain's velocity wins outweigh the framework overhead.

### Why Chroma?

When working on LLM projects that involve `semantic search` or `RAG`, vector databases are essential for storing and querying embeddings efficiently. `Chroma` stands out because it **balances ease-of-use, flexibility, and functionality**. Unlike `FAISS` (a low-level similarity search library), `Pinecone` (managed cloud service), or `Weaviate` (complex self-hosted solution), `Chroma` lets you run embedded in **your application** or as a **standalone server**, supports metadata and persistence out-of-the-box, and integrates seamlessly with frameworks like LangChain—making it ideal for developers who want to move fast without infrastructure overhead. &nbsp;&nbsp; ⇾(Expand the section for details)

**The Vector Database Landscape** <br>
Many other solutions have their trade-offs. `FAISS` is extremely fast at large-scale similarity search but requires **manual management of metadata and persistence**. `Pinecone` offers a fully managed solution with advanced features but comes with **ongoing costs** and **cloud-only** limitations. `Weaviate` provides hybrid search and ML model integration but has a **steeper learning curve** and more **complex setup**. For engineers, these hurdles can slow down project development.

**Chroma: Simplicity Without Sacrifice** <br>
`Chroma`, on the other hand, simplifies the workflow. Its Python API is intuitive, it integrates naturally with `LangChain`, and it supports both **embedded** and **client-server deployment** modes. You can store embeddings along with metadata, filter results easily, and get your RAG pipelines running quickly. For small-to-medium scale projects, it allows you to focus on building intelligent LLM applications rather than spending time on database plumbing.

**Where Chroma Makes Trade-offs** <br>
Of course, `Chroma` has limitations. It **isn't optimized for extremely large datasets** with hundreds of millions of vectors and lacks some of the **advanced indexing options** that `FAISS` offers. However, for engineers who want to rapidly prototype and deploy LLM-powered solutions, `Chroma` provides a perfect balance of simplicity, power, and flexibility to get real results without unnecessary friction.


### Implementation

In [1]:
# Adapted from "RAG + Langchain Python Project: Easy AI/Chat For Your Docs"
# https://www.youtube.com/watch?v=tcqEUSNCn8I

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_classic.prompts import ChatPromptTemplate
import openai 
from datasets import Dataset
from dotenv import load_dotenv
import os
import shutil
import numpy as np
import glob


d:\Program\GitHubProjects\Do-my-history-exam\test3.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The project begins by **converting the target documents into LangChain** `Document` **objects** using the `DirectoryLoader` function. This preserves both the **text** and **metadata**, such as the source path and start index, which are required for LangChain's downstream functions.

In [2]:
# Convert the raw files into Document
def load_documents(DATA_PATH: str):
    # loader = DirectoryLoader(DATA_PATH, glob="*.md")  # Alice in the wonderland
    # documents = loader.load()
    documents = []
    for pdf_file in glob.glob(f"{DATA_PATH}/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load()
        documents.extend(docs)

    return documents

Next, the documents are **split into smaller chunks**. This is necessary primarily for **retrieval precision in RAG systems**. Smaller chunks allow the system to retrieve **only the most relevant information** rather than entire documents, improving semantic similarity matching. **Large chunks** can **dilute semantic meaning** and introduce **irrelevant context**, while **smaller chunks** enable more **focused retrieval**. Additionally, chunking helps manage **LLM context window limitations**, though retrieval quality is the primary consideration.

The 'RecursiveCharacterTextSplitter' controls how text is divided using two key parameters: 'chunk_size' and 'chunk_overlap'. 'chunk_size' defines the *maximum length* of each chunk, while 'chunk_overlap' *repeats a portion* of the previous chunk in the next one to **reduce the chance of splitting related information** across chunk boundaries. Choosing appropriate values for these parameters directly impacts retrieval quality, and thus the overall performance of the RAG system.

In [18]:
# Split the text into small chunks
def split_text(documents: list[Document], chunk_size: int=300, chunk_overlap: int=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,          #nb of characters in each chunk
        chunk_overlap=chunk_overlap,    #nb of characters to overlap between chunks
        length_function=len,            #decide how to measure the chunk, e.g., character, token, etc
        add_start_index=True,           #add the starting index of the chunk
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} pages into {len(chunks)} chunks.\n")

    return chunks

Once the documents are split, each chunk is converted into a **vector embedding** using OpenAI's 'text-embedding-3-small' model. These **embeddings** are then stored in a 'Chroma' vector database along with the **original chunk text and metadata**. The 'Chroma' database enables efficient **similarity search** by **comparing query embeddings against stored chunk embeddings**.

It is crucial to use the **same embedding model** for both **indexing chunks** and **embedding queries** because different models produce **incompatible vector representations** that exist in different semantic spaces, making retrieval unreliable or impossible. For this project, I use the default embedding model `text-embedding-ada-002`.

In [4]:
# Apply vector embedding to chunks and save the embedding vector along with the content and metadata to database
def save_to_chroma(chunks: list[Document], CHROMA_PATH: str, model="text-embedding-ada-002"):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(
        documents=chunks, embedding=OpenAIEmbeddings(model=model), persist_directory=None
    )
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")
    return db

Main

In [5]:
# Load environment variables that contains the OpenAI API key, LangChain can access it automatically
load_dotenv()

# Set path for where to get the original file and where to safe the chunks
CHROMA_PATH = "chroma"
# DATA_PATH = "data/books"    # Alice in the wonderland
DATA_PATH = "data/papers"    # LLMs paper

# split the document in chunks and save it to database along with its embedded vector
documents = load_documents(DATA_PATH)
chunks = split_text(documents)

print(f"Print chunk 10 content: ")
document = chunks[10]
print(f"Content: \"{document.page_content}\"")
print(f"Metadata: {document.metadata}\n")

db = save_to_chroma(chunks, CHROMA_PATH)

Split 18 documents into 328 chunks.

Print chunk 10 content: 
Content: "returns a generated text completion. These input
contexts can contain thousands of tokens, espe-
cially when language models are used to process
long documents (e.g., legal or scientific documents,
conversation histories, etc.) or when language mod-"
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-11-22T01:09:40+00:00', 'author': '', 'keywords': '', 'moddate': '2023-11-22T01:09:40+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/papers\\Lost in the Middle_ How Language Models Use Long Contexts.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1', 'start_index': 1743}

Saved 328 chunks to chroma.


The **prompt template** determines how **retrieved context** and the **user query** are **structured** for the **language model**. While prompt quality is difficult to quantify precisely, adhering to **established prompting principles significantly improves model responses**. Effective LLM prompting requires **clear, specific instructions** - similar to providing detailed directions to a new team member. Well-defined prompts guide the model toward desired outputs, while vague or ambiguous instructions increase output unpredictability, often resulting in irrelevant or inaccurate responses.

In [5]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

Once the Chroma database and prompt template are configured, the system can process user queries through the following pipeline: First, the query is embedded using the **same embedding model** applied to the document chunks, ensuring vector space consistency. The system then retrieves the **top-k most similar chunks** from the vector database (using L2 (Euclidean) distance by default). These retrieved chunks are combined with the original query according to the prompt template structure and passed to the LLM (default LLM: `gpt-3.5-turbo`) for response generation.

Implementing **quality safeguards** is essential for production systems. This includes **rejecting empty** or **malformed queries** and filtering results when similarity scores fall **below a confidence threshold**, as low-similarity retrievals typically indicate insufficient relevant context and lead to unreliable outputs. With these components in place, the RAG pipeline can effectively retrieve pertinent information and generate well-informed responses.

In [10]:
# Generate the answer by feeding the LLM with prompt
def generate_answer(question: str, context: str, isprintprompt: bool=False) :
    # Generate the prompt template with context and query
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in context])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)
    if isprintprompt:
        print(prompt)

    # Implement the LLM and feed it with the prompt
    model = ChatOpenAI(model="gpt-3.5-turbo")
    return model.invoke(prompt) 

In [13]:
# query_text = input("Enter your query: ")
query_text = "How does the length of a context affect a language model’s ability to use information?"

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results, similarity too low.")
    # return

print("Top 2 similarity:")
for i, k in enumerate(results):
    if i == 2:
        break
    print(f"Top {i + 1}:\nContent: {k[0].page_content}\nL2 similarity: {k[1]}\n")

response_text = generate_answer(query_text, results, True)

# Print the formatted response
sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text.content}\nSources: {sources}"
print(formatted_response)


Unable to find matching results, similarity too low.
Top 2 similarity:
Top 1:
Content: even for explicitly long-context models. Our
analysis provides a better understanding of
how language models use their input context
and provides new evaluation protocols for
future long-context language models.
1 Introduction
Language models have become an important and
L2 similarity: 0.6654352187271577

Top 2:
Content: shows that precise knowledge access on long con-
texts may be an added challenge.
6.2 How Do Language Models Use Context?
The pioneering work of Khandelwal et al. (2018)
showed that small LSTM language models make
increasingly coarse use of longer-term context;
L2 similarity: 0.6440712551715008

Human: 
Answer the question based only on the following context:

even for explicitly long-context models. Our
analysis provides a better understanding of
how language models use their input context
and provides new evaluation protocols for
future long-context language models.
1 Introduction


### Evaluation
`RAG` evaluation breaks down into two critical stages: **retrieval** and **generation**.

**Retrieval Evaluation**<br>
In retrieval evaluation, we **measure how effectively the system identifies and ranks relevant documents for a given query**. Standard metrics like `Precision@k`, `Recall@k`, and `Mean Reciprocal Rank (MRR)` quantify whether the retriever surfaces useful chunks in its top results. High precision means most retrieved chunks are relevant; high recall means we're not missing critical information. These metrics guide optimization of chunk size, overlap, and embedding model selection—essentially tuning what context reaches the generator.

**Generation Evaluation**<br>
Generation evaluation **assesses the quality and factual grounding of responses given the retrieved context**. We use `RAGAS (Retrieval-Augmented Generation Assessment)` metrics: `faithfulness` (does the answer align with retrieved evidence?), answer correctness (how close to ground truth?), `answer relevancy` (is it on-topic?), and `context precision/recall` (did we retrieve the right supporting passages?). These metrics reveal not just what the model generates, but how well it uses retrieved knowledge.

In [7]:
# Extract content from Q&A file generated with ChatGPT
def read_qa_file_by_paper(file_path: str):
    rows = []
    current_paper = None

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # skip empty lines
            if line.lower().startswith("paper:"):
                current_paper = line[len("paper:"):].strip()
            elif line.lower().startswith("q:"):
                question = line[len("q:"):].strip()
            elif line.lower().startswith("a:"):
                answer = line[len("a:"):].strip()
                if current_paper is None:
                    raise ValueError(f"Answer found without specifying paper: {line}")
                rows.append({
                    "paper": current_paper,
                    "question": question,
                    "reference": answer
                })
            else:
                raise ValueError(f"Line format not recognized: {line}")
    return rows

In [ ]:
def generate_answer_in_eval_format(path: str, chroma: Chroma, k: int):
    evaluation_rows = []

    # Extract Q&A content from txt
    qa_rows = read_qa_file_by_paper(path)

    for row in qa_rows:
        question = row["question"]
        ground_truth = row["reference"]
        paper = row["paper"]

        # Retrieve context from database and generate the answer
        context = chroma.similarity_search_with_relevance_scores(question, k=k)
        answer = generate_answer(question, context)

        # Organize the information into the format required by RAGAS
        evaluation_rows.append({
            "question": question,
            "contexts": [doc.page_content for doc, _score in context],  # <-- plain text
            # "answer": answer_with_source,
            "answer": answer.content,
            "reference": ground_truth,
            "source": paper
        })
    return evaluation_rows

In [ ]:
# generate answer in evaluation format
qa_file = "data/papers/Beginner Q&A.txt"
evaluation_rows = generate_answer_in_eval_format(path=qa_file, chroma=db, k=2)

# change it to Dataset format
evaluation_dataset = Dataset.from_list(evaluation_rows)
print(evaluation_dataset)

Human: 
Answer the question based only on the following context:

question answering example presented in Figure 2. Re-
ordering the documents in the input context does not
affect the desired output.
Our experimental setup is similar to the needle-
in-a-haystack experiments of Ivgi et al. (2023), who
compare question answering performance when the

---

models must retrieve from the middle of the input context.
placed at the start of the input context, LongChat-
13B (16K) tends to generate code to retrieve the
key, rather than outputting the value directly.
4 Why Are Language Models Not Robust
to Changes in the Position of Relevant
Information?

---

Answer the question based on the above context: When I retrieve 5 chunks for my RAG system, does it matter what order I put them in?

Human: 
Answer the question based only on the following context:

of the context, and slightly increases performance
when using information in the middle and end of
the context.
D GPT-4 Performance
We evalua

**Why RAGAS?** <br>
Traditional NLP metrics like `BLEU` or `ROUGE` measure surface-level text similarity but ignore whether answers are grounded in evidence—critical for `RAG` systems where factual accuracy and traceability matter. `RAGAS` provides task-aware, automated evaluation that directly **measures how generated content connects to retrieved context**. Its modular design integrates seamlessly with `LangChain` and `Weights & Biases`, enabling scalable experimentation and real-time tracking. Essentially, `RAGAS` lets us **evaluate retrieval-generation alignment** systematically, leading to more interpretable and reliable `RAG` performance insights without constant human annotation.

**The limitations?** <br>
`RAGAS` relies on **LLM-as-judge evaluation**, meaning it **uses language models (often GPT-4) to score metrics** like `faithfulness` and `relevancy`. This introduces **cost** (API calls for every evaluation), **latency** (slower than traditional metrics), and **potential bias** (the judge model's own limitations affect scores). Additionally, `RAGAS` metrics **require ground truth datasets** for answer correctness, which aren't always available or easy to create for domain-specific applications. The framework also has a **learning curve**—understanding what each metric measures and how to interpret scores together requires familiarity with `RAG`-specific evaluation concepts. Despite these trade-offs, for most `RAG` applications, `RAGAS`'s ability to measure grounding and factuality outweighs the operational overhead compared to purely manual evaluation or metrics that ignore retrieval quality entirely.

In [ ]:
import numpy as np
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)

# Initialize the LLM for evaluation
llm = ChatOpenAI(model="gpt-3.5-turbo")  # sync interface compatible with RAGas

# Run evaluation
scores = evaluate(
    evaluation_dataset,
    metrics=[
        answer_correctness,
        answer_relevancy,
        faithfulness,
        context_precision,
        context_recall,
    ],
    llm=llm,  # pass the LLM explicitly
    num_samples=1
)

# Print the results
print("Evaluation dataset preview:")
for i, row in enumerate(evaluation_dataset):
    print(f"{i+1}. Q: {row['question']}")
    print(f"   Answer: {row['answer']}")
    print(f"   Reference: {row['reference']}")
    print(f"   Contexts: {row['contexts']}\n")

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  66%|██████▌   | 33/50 [01:40<00:32,  1.94s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 50/50 [02:29<00:00,  2.99s/it]


Evaluation dataset preview:
1. Q: When I retrieve 5 chunks for my RAG system, does it matter what order I put them in?
   Answer: No, re-ordering the documents in the input context does not affect the desired output in the given scenario.
   Reference: YES! The paper shows LLMs perform ~20% worse on information in the middle of context vs. beginning/end. For RAG, put your most relevant chunks at the START and END of your prompt. If you have 5 chunks ranked by relevance: [1st, 5th, 4th, 3rd, 2nd] - sandwich low-importance chunks in the middle.
   Contexts: ['question answering example presented in Figure 2. Re-\nordering the documents in the input context does not\naffect the desired output.\nOur experimental setup is similar to the needle-\nin-a-haystack experiments of Ivgi et al. (2023), who\ncompare question answering performance when the', 'models must retrieve from the middle of the input context.\nplaced at the start of the input context, LongChat-\n13B (16K) tends to generate cod

AttributeError: 'EvaluationResult' object has no attribute 'results'

In [13]:
# print RAGAS result
print(
    f"RAGAS scores:"
    f"\n Answer correctness: mean={np.mean(scores['answer_correctness']):.4f}, std={np.std(scores['answer_correctness']):.4f}"
    f"\n Answer relevancy:   mean={np.mean(scores['answer_relevancy']):.4f}, std={np.std(scores['answer_relevancy']):.4f}"
    f"\n Faithfulness:       mean={np.mean(scores['faithfulness']):.4f}, std={np.std(scores['faithfulness']):.4f}"
    f"\n Context precision:  mean={np.mean(scores['context_precision']):.4f}, std={np.std(scores['context_precision']):.4f}"
    f"\n Context recall:     mean={np.mean(scores['context_recall']):.4f}, std={np.std(scores['context_recall']):.4f}"
)

RAGAS scores:
 Answer correctness: mean=0.3471, std=0.1708
 Answer relevancy:   mean=0.6882, std=0.3478
 Faithfulness:       mean=0.6012, std=0.2875
 Context precision:  mean=0.7500, std=0.3354
 Context recall:     mean=0.3917, std=0.3576


In [ ]:
import itertools
import numpy as np
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
from datasets import Dataset
from IPython.display import display


# Load environment variables that contains the OpenAI API key, LangChain can access it automatically
load_dotenv()

# Set path for where to get the original file and where to safe the chunks
CHROMA_PATH = "chroma"
DATA_PATH = "data/papers"    # LLMs paper

# Example parameter grid
# chunk_sizes = [256, 512, 1024]
# chunk_overlaps = [0, 50]
# embedding_models = ["text-embedding-3-small", "text-embedding-3-large"]
# retriever_top_k = [2, 5]

chunk_sizes = [300]
chunk_overlaps = [0, 50, 100]
embedding_models = ["text-embedding-3-small"]
retriever_top_k = [2]

results = []
documents = load_documents(DATA_PATH)

for chunk_size, overlap, emb_model, top_k in itertools.product(chunk_sizes, chunk_overlaps, embedding_models, retriever_top_k):
    print(f"\n🔧 Running config: chunk={chunk_size}, overlap={overlap}, model={emb_model}, top_k={top_k}")

    # --- Step 1: Build your retriever & RAG pipeline ---
    chunks = split_text(documents, chunk_size, overlap)
    db = save_to_chroma(chunks, CHROMA_PATH, emb_model)

    # generate answer in evaluation format and change it to dataset
    qa_file = "data/papers/Beginner Q&A_eval.txt"
    evaluation_rows = generate_answer_in_eval_format(qa_file, db, top_k)
    evaluation_dataset = Dataset.from_list(evaluation_rows)

    # sync interface compatible with RAGas
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # Explicitly pass to model kwargs)  

    # Run evaluation
    scores = evaluate(
        evaluation_dataset,
        metrics=[
            answer_correctness,
            answer_relevancy,
            faithfulness,
            context_precision,
            context_recall,
        ],
        llm=llm,  # pass the LLM explicitly
    )

    # --- Step 3: Collect results ---
    results.append({
        "chunk_size": chunk_size,
        "chunk_overlap": overlap,
        "embedding_model": emb_model,
        "top_k": top_k,
        "answer_correctness_mean": np.mean(scores["answer_correctness"]),
        "answer_correctness_std": np.std(scores["answer_correctness"]),
        "answer_relevancy_mean": np.mean(scores["answer_relevancy"]),
        "answer_relevancy_std": np.std(scores["answer_relevancy"]),
        "faithfulness_mean": np.mean(scores["faithfulness"]),
        "faithfulness_std": np.std(scores["faithfulness"]),
        "context_precision_mean": np.mean(scores["context_precision"]),
        "context_precision_std": np.std(scores["context_precision"]),
        "context_recall_mean": np.mean(scores["context_recall"]),
        "context_recall_std": np.std(scores["context_recall"]),
    })

# --- Step 4: Save and inspect ---
df = pd.DataFrame(results)
df.to_csv("ragas_auto_eval_results.csv", index=False)
print("\n✅ Evaluation complete! Results saved to ragas_auto_eval_results.csv")
display(df)



🔧 Running config: chunk=300, overlap=0, model=text-embedding-3-small, top_k=2
Split 18 pages into 244 chunks.

Saved 244 chunks to chroma.


ValidationError: 1 validation error for ChatOpenAI
  Value error, Found n supplied twice. [type=value_error, input_value={'model': 'gpt-3.5-turbo'...model_kwargs': {'n': 3}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error